In [1]:
import numpy as np
from pynq import Overlay, allocate

np.set_printoptions(suppress=True)

In [2]:
x = np.loadtxt("wave.txt", dtype=np.float32)
TWO_TO_THE_POWER_OF_N_MAX = 10
NFFT = 1 << TWO_TO_THE_POWER_OF_N_MAX

In [3]:
overlay = Overlay("DMA_FFT_wrapper_10032023.bit")
in_buffer = allocate(shape=(NFFT,), dtype=np.csingle)
out_buffer = allocate(shape=(NFFT,), dtype=np.csingle)
data = x[:NFFT].astype(np.csingle)

fft_ip = overlay.hls_fft_0
dma_ip = overlay.axi_dma_0

fft_ip.write(0x10, TWO_TO_THE_POWER_OF_N_MAX)

In [4]:
data[1]

(0.7616634+0j)

In [5]:
for i in range(NFFT):
        in_buffer[i] = data[i]
in_buffer        

PynqBuffer([0.       +0.j, 0.7616634+0.j, 1.4614664+0.j, ...,
            1.1911113+0.j, 1.554413 +0.j, 1.7994008+0.j], dtype=complex64)

In [6]:
def fft_on_hw(fft_ip, dma_ip, input_data):    
    
    for i in range(NFFT):
        in_buffer[i] = input_data[i]
        
    dma_ip.sendchannel.transfer(in_buffer)
    dma_ip.recvchannel.transfer(out_buffer)
    fft_ip.write(0x00, 0x01)
    while not ((fft_ip.read(0x00) >> 1) & 0x01):
        continue
    # dma_ip.sendchannel.wait()
    # dma_ip.recvchannel.wait() 
    return out_buffer.copy()

In [7]:
fft_on_hw(fft_ip, dma_ip, data)

PynqBuffer([11.942313+0.j        , 11.952061-0.15409547j,
            11.981426-0.3086664j , ..., 12.030692+0.46418774j,
            11.981438+0.30866188j, 11.952067+0.15408981j], dtype=complex64)

In [8]:
out_buffer

PynqBuffer([11.942313+0.j        , 11.952061-0.15409547j,
            11.981426-0.3086664j , ..., 12.030692+0.46418774j,
            11.981438+0.30866188j, 11.952067+0.15408981j], dtype=complex64)